In [2]:
# !git clone https://github.com/aditya-grover/node2vec.git

In [3]:
# !python node2vec/src/main.py --input diseasome.edgelist --output diseasome.emb --dimensions 32 --num-walks 20 --iter 10

In [4]:
# !pip install torch

In [5]:
import torch

In [6]:
%matplotlib inline

import csv

from matplotlib import pyplot as plt
from sklearn import manifold
import pandas as pd
import numpy as np

In [7]:
# node2emb = {}
# with open('diseasome.emb') as fin:
#     fin.readline()
#     for line in fin:
#         node_emb = line.strip().split()
#         node2emb[node_emb[0]] = np.array(node_emb[1:]).astype(float)

# nodes, embs = zip(*node2emb.items())
# embs = np.array(embs)

In [8]:
# node2name_label = {}
# with open('disease.labels') as fin:
#     fin.readline()
#     csvreader = csv.reader(fin)
#     for line in csvreader:
#         nid, name, label = line
#         node2name_label[nid] = name, label

# labels = [node2name_label[nid][1] for nid in nodes]
# all_labels = list(set(labels))

In [9]:
# len(labels), len(all_labels), embs.shape

In [10]:
# tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
# proj = tsne.fit_transform(embs)

In [11]:
# df = pd.DataFrame(dict(x=proj[:, 0], y=proj[:, 1], label=labels))
# groups = df.groupby('label')

# nr, nc = 4, 5
# ii = 0
# fig, axes = plt.subplots(nr, nc, figsize=(16, 10))
# for r in range(nr):
#     for c in range(nc):
#         for name, group in groups:
#             col, z, a = ('red', 10, 1) if name == all_labels[ii] else ('black', 1, 0.4)
#             axes[r, c].plot(group.x, group.y, marker='o', linestyle='', ms=5, color=col, zorder=z, alpha=a)
#         axes[r, c].set_title(all_labels[ii])
#         ii += 1

# plt.tight_layout()

In [12]:
import networkx as nx

In [ ]:
G = nx.read_edgelist("diseasome.edgelist")
df_adj = nx.to_pandas_adjacency(G)

In [14]:
print(
    f"The number of connected diseases = {round(df_adj.sum().sum() / (df_adj.shape[0] * df_adj.shape[1]) * 100, 3)} %"
)
print(
    f"The number of diseases in data = {df_adj.shape[0]}"
)

The number of connected diseases = 0.892 %
The number of diseases in data = 516


In [15]:
disease_labels_df = pd.read_csv("disease.labels")
disease_labels_df

,Id,Label,Disease class
0,222,Osteoporosis,Bone
1,306,Epiphyseal dysplasia,Bone
2,277,Osteopetrosis,Bone
3,286,Osteogenesis imperfecta,Bone
4,288,"Achondrogenesis-hypochondrogenesis, type II",Bone
...,...,...,...
511,643,Alcohol dependence,Unclassified
512,747,van Buchem disease,Unclassified
513,842,Placental abruption,Unclassified
514,941,Bannayan-Riley-Ruvalcaba syndrome,Unclassified


In [16]:
all_present_label_symbols = set(";".join(disease_labels_df["Label"].values.tolist()))
alphabetic_symbols = set([chr(i) for i in range(ord("A"), ord("Z") + 1)] + [chr(i) for i in range(ord("a"), ord("z") + 1)])

len(all_present_label_symbols), len(alphabetic_symbols)

(62, 52)

In [17]:
not_alphabetic_symbols = list(all_present_label_symbols.difference(alphabetic_symbols))
len(not_alphabetic_symbols), not_alphabetic_symbols

(10, ["'", '3', '/', ' ', '2', '-', '6', ';', ',', '1'])

In [21]:
# !pip install simple_icd_10

In [22]:
import simple_icd_10 as icd

In [23]:
def find_icd_by_name(target_name, not_alphabetic_symbols):
    target_name = target_name.lower()
    for el in not_alphabetic_symbols:
        target_name = target_name.replace(el, "")
    found_codes = []
    for code in icd.get_all_codes():
        if "." in code:
            description = icd.get_description(code).lower()
            for el in not_alphabetic_symbols:
                description = description.replace(el, "")
            if target_name in description or description in target_name:
                found_codes.append(icd.get_parent(code))

    if len(found_codes) > 0:
        return list(set(found_codes))
    return np.nan

In [24]:
disease_labels_df["Possible ICDs"] = disease_labels_df["Label"].apply(lambda x: find_icd_by_name(x, not_alphabetic_symbols))
disease_labels_df

,Id,Label,Disease class,Possible ICDs
0,222,Osteoporosis,Bone,"[M82, M81, M80]"
1,306,Epiphyseal dysplasia,Bone,[Q77]
2,277,Osteopetrosis,Bone,[Q78]
3,286,Osteogenesis imperfecta,Bone,[Q78]
4,288,"Achondrogenesis-hypochondrogenesis, type II",Bone,[Q77]
...,...,...,...,...
511,643,Alcohol dependence,Unclassified,NaN
512,747,van Buchem disease,Unclassified,NaN
513,842,Placental abruption,Unclassified,NaN
514,941,Bannayan-Riley-Ruvalcaba syndrome,Unclassified,NaN


In [25]:
disease_labels_df["Possible ICDs"].value_counts()

Possible ICDs
[Q77]                   6
[G71]                   4
[G72]                   3
[F02]                   3
[F84]                   3
                       ..
[T48, Z82, Y55, J45]    1
[J98, P25, T79, J43]    1
[Q75]                   1
[Q70]                   1
[G56]                   1
Name: count, Length: 107, dtype: int64

In [26]:
disease_labels_df[disease_labels_df["Possible ICDs"].isna()]

,Id,Label,Disease class,Possible ICDs
5,311,OSMED syndrome,Bone,NaN
6,405,"Dentin dysplasia, type II",Bone,NaN
7,406,"Dentinogenesis imperfecta, Shields type",Bone,NaN
9,577,Osseous heteroplasia,Bone,NaN
10,637,"Apparent mineralocorticoid excess, hypertensio...",Bone,NaN
...,...,...,...,...
510,380,Benzene toxicity,Unclassified,NaN
511,643,Alcohol dependence,Unclassified,NaN
512,747,van Buchem disease,Unclassified,NaN
513,842,Placental abruption,Unclassified,NaN


In [27]:
"""
Enhanced ICD-10 Disease Mapper using Semantic Similarity
Uses BERT embeddings to find the best matching ICD-10 codes for disease names
"""

import pandas as pd
import numpy as np
import re
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Try to import required libraries
try:
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    HAS_BERT = True
except ImportError:
    HAS_BERT = False
    print("Warning: sentence-transformers not installed. Install with: pip install sentence-transformers")

try:
    import simple_icd_10 as icd
    HAS_ICD = True
except ImportError:
    HAS_ICD = False
    print("Warning: simple_icd_10 not installed. Install with: pip install simple-icd-10")


def clean_text(text: str, remove_symbols: List[str] = None) -> str:
    """Clean and normalize text for better matching"""
    if remove_symbols is None:
        remove_symbols = [',', '.', '-', '(', ')', '[', ']', '/', '\\', ':', ';', "'"]
    
    text = text.lower()
    for symbol in remove_symbols:
        text = text.replace(symbol, ' ')
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text


def normalize_disease_name(disease_name: str) -> str:
    """
    Normalize disease names by removing common suffixes and prefixes
    that might prevent exact matching
    """
    name = disease_name.lower()
    
    # Common patterns to try removing
    patterns_to_remove = [
        r',?\s+type\s+[ivxIVX0-9]+\s*$',  # Type I, Type II, etc.
        r',?\s+type\s+[a-zA-Z]+\s*$',      # Type A, Type B, etc.
        r'\s+syndrome\s*$',                 # syndrome at end
        r'\s+disease\s*$',                  # disease at end
        r'^\s*familial\s+',                 # familial at start
        r'^\s*hereditary\s+',               # hereditary at start
        r'^\s*congenital\s+',               # congenital at start
        r',.*$',                            # Everything after comma
    ]
    variations = [name]
    
    for pattern in patterns_to_remove:
        variant = re.sub(pattern, '', name).strip()
        if variant and variant != name:
            variations.append(variant)
    
    return variations


def find_icd_by_name_exact(target_name: str, not_alphabetic_symbols: List[str]) -> List[str]:
    """Original exact matching function (improved)"""
    if not HAS_ICD:
        return []
    
    target_clean = clean_text(target_name, not_alphabetic_symbols)
    found_codes = []
    
    for code in icd.get_all_codes():
        if "." in code:
            description = clean_text(icd.get_description(code), not_alphabetic_symbols)
            
            # Check both directions with cleaned text
            if target_clean in description or description in target_clean:
                parent_code = icd.get_parent(code)
                found_codes.append(parent_code)
    
    if len(found_codes) > 0:
        return list(set(found_codes))
    
    return []


def find_icd_by_name_variants(target_name: str, not_alphabetic_symbols: List[str]) -> List[str]:
    """Try matching with normalized variants of the disease name"""
    if not HAS_ICD:
        return []
    
    variants = normalize_disease_name(target_name)
    found_codes = []
    
    for variant in variants:
        codes = find_icd_by_name_exact(variant, not_alphabetic_symbols)
        if codes:
            found_codes.extend(codes)
    
    if found_codes:
        return list(set(found_codes))
    
    return []


def build_icd_database() -> Tuple[List[str], List[str], np.ndarray]:
    """Build database of all ICD-10 codes and their descriptions with embeddings"""
    if not HAS_ICD:
        return [], [], np.array([])
    
    codes = []
    descriptions = []
    
    print("Building ICD-10 code database...")
    for code in icd.get_all_codes():
        if "." in code:
            parent_code = icd.get_parent(code)
            desc = icd.get_description(code)
            
            # Store unique parent codes
            if parent_code not in codes:
                codes.append(parent_code)
                descriptions.append(desc)
    
    print(f"Found {len(codes)} unique 3-character ICD-10 codes")
    
    # Create embeddings if BERT is available
    embeddings = None
    if HAS_BERT:
        print("Creating BERT embeddings for ICD-10 descriptions...")
        model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')  # Faster than bert-base
        embeddings = model.encode(descriptions, show_progress_bar=True)
        print(f"Embeddings shape: {embeddings.shape}")
    
    return codes, descriptions, embeddings


def find_icd_by_semantic_similarity(
    disease_name: str,
    icd_codes: List[str],
    icd_descriptions: List[str],
    icd_embeddings: np.ndarray,
    model: SentenceTransformer,
    top_k: int = 3,
    threshold: float = 0.5
) -> List[Tuple[str, float]]:
    """
    Find ICD-10 codes using semantic similarity with BERT embeddings
    
    Returns list of (code, similarity_score) tuples
    """
    if not HAS_BERT or icd_embeddings is None:
        return []
    
    # Encode the disease name
    disease_embedding = model.encode([disease_name])
    
    # Calculate cosine similarity
    similarities = cosine_similarity(disease_embedding, icd_embeddings)[0]
    
    # Get top-k matches above threshold
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    
    results = []
    for idx in top_indices:
        if similarities[idx] >= threshold:
            results.append((icd_codes[idx], float(similarities[idx])))
    
    return results


def map_diseases_to_icd10(
    disease_file: str,
    output_file: str = 'disease_icd10_mapping_enhanced.csv',
    use_bert: bool = True,
    bert_threshold: float = 0.5
):
    """
    Main function to map diseases to ICD-10 codes using multiple strategies
    """
    # Read disease file
    print(f"Reading diseases from {disease_file}...")
    df = pd.read_csv(disease_file)
    
    not_alphabetic_symbols = [',', '.', '-', '(', ')', '[', ']', '/', '\\', ':', ';']
    
    # Build ICD-10 database
    icd_codes, icd_descriptions, icd_embeddings = build_icd_database()
    
    # Initialize BERT model if requested
    model = None
    if use_bert and HAS_BERT:
        print("Loading BERT model...")
        model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
    # Process each disease
    results = []
    
    print(f"\nProcessing {len(df)} diseases...")
    for idx, row in df.iterrows():
        disease_id = row['Id']
        disease_name = row['Label']
        disease_class = row['Disease class']
        
        mapped_codes = []
        mapping_method = 'unmapped'
        similarity_score = None
        
        # Strategy 1: Exact match
        codes = find_icd_by_name_exact(disease_name, not_alphabetic_symbols)
        if codes:
            mapped_codes = codes
            mapping_method = 'exact_match'
        
        # Strategy 2: Match with normalized variants
        if not mapped_codes:
            codes = find_icd_by_name_variants(disease_name, not_alphabetic_symbols)
            if codes:
                mapped_codes = codes
                mapping_method = 'variant_match'
        
        # Strategy 3: BERT semantic similarity
        if not mapped_codes and use_bert and model is not None:
            semantic_matches = find_icd_by_semantic_similarity(
                disease_name,
                icd_codes,
                icd_descriptions,
                icd_embeddings,
                model,
                top_k=3,
                threshold=bert_threshold
            )
            
            if semantic_matches:
                # Take the top match
                mapped_codes = [semantic_matches[0][0]]
                similarity_score = semantic_matches[0][1]
                mapping_method = f'bert_similarity'
                
                # Show all top matches for review
                all_matches = '; '.join([f"{code}({score:.3f})" for code, score in semantic_matches])
                mapping_method += f' [alternatives: {all_matches}]'
        
        # Store result
        result = {
            'disease_id': disease_id,
            'disease_name': disease_name,
            'disease_class': disease_class,
            'icd10_codes': '; '.join(mapped_codes) if mapped_codes else None,
            'primary_icd10': mapped_codes[0] if mapped_codes else None,
            'mapping_method': mapping_method,
            'similarity_score': similarity_score
        }
        results.append(result)
        
        # Print progress
        if mapped_codes:
            print(f"✓ [{idx+1}/{len(df)}] {disease_name} → {mapped_codes[0]} ({mapping_method})")
        else:
            print(f"✗ [{idx+1}/{len(df)}] {disease_name} - NO MATCH")
    
    # Create results dataframe
    results_df = pd.DataFrame(results)
    
    # Save results
    results_df.to_csv(output_file, index=False)
    print(f"\nResults saved to {output_file}")
    
    # Print summary statistics
    print("\n" + "="*70)
    print("MAPPING SUMMARY:")
    print("="*70)
    
    total = len(results_df)
    mapped = results_df['primary_icd10'].notna().sum()
    unmapped = total - mapped
    
    print(f"Total diseases: {total}")
    print(f"Mapped: {mapped} ({100*mapped/total:.1f}%)")
    print(f"Unmapped: {unmapped} ({100*unmapped/total:.1f}%)")
    
    # Breakdown by method
    print("\nMapping method breakdown:")
    method_counts = results_df['mapping_method'].value_counts()
    for method, count in method_counts.items():
        if method != 'unmapped':
            # Clean method name for display
            clean_method = method.split('[')[0].strip()
            print(f"  {clean_method}: {count}")
    
    print("="*70)
    
    # Save unmapped diseases separately
    unmapped_df = results_df[results_df['primary_icd10'].isna()]
    if len(unmapped_df) > 0:
        unmapped_file = output_file.replace('.csv', '_unmapped.csv')
        unmapped_df.to_csv(unmapped_file, index=False)
        print(f"\nUnmapped diseases saved to {unmapped_file}")
    
    return results_df, unmapped_df


if not HAS_ICD:
    print("\nERROR: simple_icd_10 not installed!")
    print("Install with: pip install simple-icd-10")
    exit(1)

if not HAS_BERT:
    print("\nWARNING: sentence-transformers not installed!")
    print("Install with: pip install sentence-transformers")
    print("Proceeding without BERT semantic matching...\n")

# Run the mapping
results_df, unmapped_df = map_diseases_to_icd10(
    disease_file='disease.labels',
    output_file='disease_icd10_mapping_enhanced.csv',
    use_bert=HAS_BERT,  # Use BERT if available
    bert_threshold=0.5   # Adjust this to be more/less strict (0.3-0.7 recommended)
)

print("\n" + "="*70)
print("DONE!")
print("="*70)
print("\nOutput files:")
print("  1. disease_icd10_mapping_enhanced.csv - All diseases with mappings")
print("  2. disease_icd10_mapping_enhanced_unmapped.csv - Unmapped diseases only")
print("\nYou can review the 'mapping_method' column to see how each disease was mapped.")
print("For BERT matches, alternatives are shown in brackets for manual review.")

Reading diseases from disease.labels...
Building ICD-10 code database...
Found 1588 unique 3-character ICD-10 codes
Creating BERT embeddings for ICD-10 descriptions...


Batches: 100%|██████████| 50/50 [00:01<00:00, 25.50it/s]


Embeddings shape: (1588, 384)
Loading BERT model...

Processing 516 diseases...
✓ [1/516] Osteoporosis → M82 (exact_match)
✓ [2/516] Epiphyseal dysplasia → Q77 (exact_match)
✓ [3/516] Osteopetrosis → Q78 (exact_match)
✓ [4/516] Osteogenesis imperfecta → Q78 (exact_match)
✓ [5/516] Achondrogenesis-hypochondrogenesis, type II → Q77 (exact_match)
✓ [6/516] OSMED syndrome → M15 (bert_similarity [alternatives: M15(0.567); Q78(0.544); M05(0.524)])
✓ [7/516] Dentin dysplasia, type II → K07 (bert_similarity [alternatives: K07(0.504)])
✗ [8/516] Dentinogenesis imperfecta, Shields type - NO MATCH
✓ [9/516] Achondrogenesis Ib → Q77 (exact_match)
✓ [10/516] Osseous heteroplasia → N85 (bert_similarity [alternatives: N85(0.531); M15(0.526)])
✓ [11/516] Apparent mineralocorticoid excess, hypertension due to → Y54 (bert_similarity [alternatives: Y54(0.715); T50(0.634); R79(0.633)])
✓ [12/516] Bone mineral density variability → R79 (bert_similarity [alternatives: R79(0.504)])
✓ [13/516] Hyperostosis, e

In [28]:
df_mapped = pd.read_csv("disease_icd10_mapping_enhanced.csv")
df_mapped

,disease_id,disease_name,disease_class,icd10_codes,primary_icd10,mapping_method,similarity_score
0,222,Osteoporosis,Bone,M82; M81; M80,M82,exact_match,NaN
1,306,Epiphyseal dysplasia,Bone,Q77,Q77,exact_match,NaN
2,277,Osteopetrosis,Bone,Q78,Q78,exact_match,NaN
3,286,Osteogenesis imperfecta,Bone,Q78,Q78,exact_match,NaN
4,288,"Achondrogenesis-hypochondrogenesis, type II",Bone,Q77,Q77,exact_match,NaN
...,...,...,...,...,...,...,...
511,643,Alcohol dependence,Unclassified,F10,F10,exact_match,NaN
512,747,van Buchem disease,Unclassified,NaN,NaN,unmapped,NaN
513,842,Placental abruption,Unclassified,O45,O45,bert_similarity [alternatives: O45(0.639); O73...,0.639187
514,941,Bannayan-Riley-Ruvalcaba syndrome,Unclassified,J43,J43,bert_similarity [alternatives: J43(0.503)],0.503033


In [33]:
df_mapped["similarity_score"].isna().sum(), sum(df_mapped["similarity_score"] > 0.5), sum(df_mapped["similarity_score"] >= 0.5), sum(df_mapped["similarity_score"] < 0.5)

(np.int64(265), 251, 251, 0)

In [34]:
df_mapped["mapping_method"].value_counts()

mapping_method
exact_match                                                           127
unmapped                                                              121
variant_match                                                          17
bert_similarity [alternatives: J43(0.554)]                              2
bert_similarity [alternatives: K07(0.504)]                              1
                                                                     ... 
bert_similarity [alternatives: N87(0.611); N89(0.611); N90(0.602)]      1
bert_similarity [alternatives: O41(0.630); K00(0.535); R60(0.510)]      1
bert_similarity [alternatives: T52(0.619); T65(0.607); T53(0.594)]      1
bert_similarity [alternatives: O45(0.639); O73(0.635); O43(0.634)]      1
bert_similarity [alternatives: J43(0.503)]                              1
Name: count, Length: 253, dtype: int64

In [41]:
516 - (127 + 121 + 17)

251

In [40]:
df_mapped[~df_mapped["mapping_method"].isin(["exact_match", "unmapped", "variant_match"])]

,disease_id,disease_name,disease_class,icd10_codes,primary_icd10,mapping_method,similarity_score
5,311,OSMED syndrome,Bone,M15,M15,bert_similarity [alternatives: M15(0.567); Q78...,0.566977
6,405,"Dentin dysplasia, type II",Bone,K07,K07,bert_similarity [alternatives: K07(0.504)],0.504121
9,577,Osseous heteroplasia,Bone,N85,N85,bert_similarity [alternatives: N85(0.531); M15...,0.530981
10,637,"Apparent mineralocorticoid excess, hypertensio...",Bone,Y54,Y54,bert_similarity [alternatives: Y54(0.715); T50...,0.714755
11,743,Bone mineral density variability,Bone,R79,R79,bert_similarity [alternatives: R79(0.504)],0.503950
...,...,...,...,...,...,...,...
501,553,Oculodentodigital dysplasia,Skeletal,N87,N87,bert_similarity [alternatives: N87(0.611); N89...,0.611405
505,879,Oligodontia,Skeletal,O41,O41,bert_similarity [alternatives: O41(0.630); K00...,0.629544
510,380,Benzene toxicity,Unclassified,T52,T52,bert_similarity [alternatives: T52(0.619); T65...,0.618823
513,842,Placental abruption,Unclassified,O45,O45,bert_similarity [alternatives: O45(0.639); O73...,0.639187


In [38]:
df_unmapped = pd.read_csv("disease_icd10_mapping_enhanced_unmapped.csv")
df_unmapped

,disease_id,disease_name,disease_class,icd10_codes,primary_icd10,mapping_method,similarity_score
0,406,"Dentinogenesis imperfecta, Shields type",Bone,NaN,NaN,unmapped,NaN
1,384,Glioblastoma,Cancer,NaN,NaN,unmapped,NaN
2,203,Wilms tumor,Cancer,NaN,NaN,unmapped,NaN
3,497,Rhabdomyosarcoma,Cancer,NaN,NaN,unmapped,NaN
4,118,Turcot syndrome,Cancer,NaN,NaN,unmapped,NaN
...,...,...,...,...,...,...,...
116,877,Morning glory disc anomaly,Skeletal,NaN,NaN,unmapped,NaN
117,991,Solitary median maxillary central incisor,Skeletal,NaN,NaN,unmapped,NaN
118,69,"Beta-2-adrenoreceptor agonist, reduced respons...",Unclassified,NaN,NaN,unmapped,NaN
119,134,Aquaporin-1 deficiency,Unclassified,NaN,NaN,unmapped,NaN


In [43]:
df_unmapped["disease_name"].values.tolist()

['Dentinogenesis imperfecta, Shields type',
 'Glioblastoma',
 'Wilms tumor',
 'Rhabdomyosarcoma',
 'Turcot syndrome',
 'Squamous cell carcinoma',
 'Pheochromocytoma',
 'Cancer susceptibility',
 'Cafe-au-lait spots',
 'Li-Fraumeni syndrome',
 'Gardner syndrome',
 'Li Fraumeni syndrome',
 'Lynch cancer family syndrome II',
 'Lhermitte-Duclos syndrome',
 'Merkel cell carcinoma',
 'Peutz-Jeghers syndrome',
 'Oligodontia-colorectal cancer syndrome',
 'Long QT syndrome',
 'Acquired long QT syndrome',
 'Brugada syndrome',
 'Noncompaction of left ventricular myocardium',
 'Wolff-Parkinson-White syndrome',
 'Weill-Marchesani syndrome',
 'Weissenbacher-Zweymuller syndrome',
 'EBD',
 'Piebaldism',
 'Unna-Thost disease, nonepidermolytic',
 'Self-healing collodion baby',
 'Netherton syndrome',
 'Birt-Hogg-Dube syndrome',
 'Peters anomaly',
 'Angelman syndrome',
 'Leprechaunism',
 'MODY',
 'Growth hormone',
 'Goiter',
 'Angiotensin I-converting enzyme',
 'Total iodide organication defect',
 'Sea-blu

In [44]:
dict_qwen = {
  "Dentinogenesis imperfecta, Shields type": "K00",
  "Glioblastoma": "C71",
  "Wilms tumor": "C64",
  "Rhabdomyosarcoma": "C49",
  "Turcot syndrome": "Q85",
  "Squamous cell carcinoma": "C44",
  "Pheochromocytoma": "C74",
  "Cancer susceptibility": "Z15",
  "Cafe-au-lait spots": "L81",
  "Li-Fraumeni syndrome": "Z15",
  "Gardner syndrome": "D12",
  "Li Fraumeni syndrome": "Z15",
  "Lynch cancer family syndrome II": "Z15",
  "Lhermitte-Duclos syndrome": "D33",
  "Merkel cell carcinoma": "C4A",
  "Peutz-Jeghers syndrome": "Q85",
  "Oligodontia-colorectal cancer syndrome": "Z15",
  "Long QT syndrome": "I45",
  "Acquired long QT syndrome": "I45",
  "Brugada syndrome": "I49",
  "Noncompaction of left ventricular myocardium": "I42",
  "Wolff-Parkinson-White syndrome": "I45",
  "Weill-Marchesani syndrome": "Q87",
  "Weissenbacher-Zweymuller syndrome": "Q77",
  "EBD": "Q81",
  "Piebaldism": "L81",
  "Unna-Thost disease, nonepidermolytic": "Q82",
  "Self-healing collodion baby": "Q80",
  "Netherton syndrome": "Q80",
  "Birt-Hogg-Dube syndrome": "Q87",
  "Peters anomaly": "Q13",
  "Angelman syndrome": "Q93",
  "Leprechaunism": "E34",
  "MODY": "E13",
  "Growth hormone": "E23",
  "Goiter": "E04",
  "Angiotensin I-converting enzyme": "Z83",
  "Total iodide organication defect": "E03",
  "Sea-blue histiocyte disease": "D76",
  "Coumarin resistance": "D68",
  "Hematopoiesis, cyclic": "D70",
  "Adult i phenotype": "D75",
  "May-Hegglin anomaly": "D69",
  "Sebastian syndrome": "D69",
  "Pyropoikilocytosis": "D58",
  "Complementary component deficiency": "D84",
  "Atopy": "L20",
  "Adenosine deaminase deficiency": "D81",
  "Graft-versus-host disease": "T86",
  "IgE levels QTL": "Z15",
  "Insulin resistance": "E88",
  "HDL cholesterol level QTL": "E78",
  "Homocystinuria": "E72",
  "Tangier disease": "E78",
  "Hypobetalipoproteinemia": "E78",
  "Placental steroid sulfatase deciency": "E88",
  "Creatine phosphokinase": "R74",
  "Dopamine beta-hydroxylase deciency": "E70",
  "Favism": "D55",
  "Glutathione synthetase deciency": "D55",
  "Sandho disease, infantile, juvenile, and adult forms": "E75",
  "3-methylglutaconicaciduria": "E71",
  "Fanconi anemia": "D61",
  "Usher syndrome": "H35",
  "Mitochondrial complex deciency": "E88",
  "Rubenstein-Taybi syndrome": "Q87",
  "Alexander disease": "G37",
  "Vohwinkel syndrome": "Q82",
  "Jervell and Lange-Nielsen syndrome": "I45",
  "Roussy-Levy syndrome": "G60",
  "Carney complex": "Q87",
  "Occipital horn syndrome": "E83",
  "Williams-Beuren syndrome": "Q93",
  "Shprintzen-Goldberg syndrome": "Q87",
  "Simpson-Golabi-Behmel syndrome": "Q87",
  "Costello syndrome": "Q87",
  "Nijmegen breakage syndrome": "D82",
  "Zlotogora-Ogur syndrome": "Q87",
  "Parkes Weber syndrome": "Q27",
  "Coffin-Lowry syndrome": "Q87",
  "Immunodysregulation, polyendocrinopathy, and enteropathy, X-linked": "D82",
  "Myotonia congenita": "G71",
  "Central core disease": "G71",
  "Cramps, potassium-aggravated": "G71",
  "Paramyotonia congenita": "G71",
  "Charcot-Marie-Tooth disease": "G60",
  "Leigh syndrome": "G31",
  "Subcortical laminar heterotopia": "Q04",
  "Branchiootic syndrome": "Q18",
  "Pallidopontonigral degeneration": "G23",
  "PPM-X syndrome": "Q99",
  "Norrie disease": "Q15",
  "Pelizaeus-Merzbacher disease": "E75",
  "Gerstmann-Straussler disease": "A81",
  "Prion disease with protracted course": "A81",
  "Creatine deciency syndrome, X-linked": "E72",
  "PCWH": "Q87",
  "Leanness, inherited": "E88",
  "Leber congenital amaurosis": "H35",
  "Iridogoniodysgenesis": "Q13",
  "Wagner syndrome": "H35",
  "Axenfeld anomaly": "Q13",
  "Coats disease": "H35",
  "Foveal hypoplasia": "Q14",
  "Butterfly dystrophy, retinal": "H35",
  "Foveomacular dystrophy, adult-onset, with choroidal neovascularization": "H35",
  "Bothnia retinal dystrophy": "H35",
  "Liddle syndrome": "I15",
  "Denys-Drash syndrome": "Q60",
  "Central hypoventilation syndrome": "G47",
  "Pfeiffer syndrome": "Q75",
  "Parietal foramina": "Q75",
  "SMED Strudwick type": "Q77",
  "Thanatophoric dysplasia, types I and II": "Q77",
  "Larson syndrome": "Q74",
  "Vertical talus": "Q66",
  "Morning glory disc anomaly": "Q14",
  "Solitary median maxillary central incisor": "K00",
  "Beta-2-adrenoreceptor agonist, reduced response to": "Z92",
  "Aquaporin-1 deficiency": "D58",
  "van Buchem disease": "Q78"
}
dict_gemma = {
  "Dentinogenesis imperfecta, Shields type": "K00",
  "Glioblastoma": "C71",
  "Wilms tumor": "C64",
  "Rhabdomyosarcoma": "C49",
  "Turcot syndrome": "Q85",
  "Squamous cell carcinoma": "C44",
  "Pheochromocytoma": "C74",
  "Cancer susceptibility": "Z15",
  "Cafe-au-lait spots": "L81",
  "Li-Fraumeni syndrome": "Z15",
  "Gardner syndrome": "D12",
  "Li Fraumeni syndrome": "Z15",
  "Lynch cancer family syndrome II": "Z15",
  "Lhermitte-Duclos syndrome": "Q04",
  "Merkel cell carcinoma": "C4A",
  "Peutz-Jeghers syndrome": "Q85",
  "Oligodontia-colorectal cancer syndrome": "K00",
  "Long QT syndrome": "I49",
  "Acquired long QT syndrome": "I49",
  "Brugada syndrome": "I49",
  "Noncompaction of left ventricular myocardium": "I42",
  "Wolff-Parkinson-White syndrome": "I49",
  "Weill-Marchesani syndrome": "Q87",
  "Weissenbacher-Zweymuller syndrome": "Q87",
  "EBD": "Q82",
  "Piebaldism": "Q82",
  "Unna-Thost disease, nonepidermolytic": "L85",
  "Self-healing collodion baby": "Q80",
  "Netherton syndrome": "Q82",
  "Birt-Hogg-Dube syndrome": "Q87",
  "Peters anomaly": "Q13",
  "Angelman syndrome": "Q93",
  "Leprechaunism": "E74",
  "MODY": "E13",
  "Growth hormone": "E23",
  "Goiter": "E04",
  "Angiotensin I-converting enzyme": np.nan,
  "Total iodide organication defect": "E07",
  "Sea-blue histiocyte disease": "D76",
  "Coumarin resistance": "D68",
  "Hematopoiesis, cyclic": "D61",
  "Adult i phenotype": "D59",
  "May-Hegglin anomaly": "D72",
  "Sebastian syndrome": "D72",
  "Pyropoikilocytosis": "D55",
  "Complementary component deficiency": "D84",
  "Atopy": "L20",
  "Adenosine deaminase deficiency": "D81",
  "Graft-versus-host disease": "T86",
  "IgE levels QTL": np.nan,
  "Insulin resistance": "E88",
  "HDL cholesterol level QTL": np.nan,
  "Homocystinuria": "E72",
  "Tangier disease": "E78",
  "Hypobetalipoproteinemia": "E78",
  "Placental steroid sulfatase deciency": "E70",
  "Creatine phosphokinase": np.nan,
  "Dopamine beta-hydroxylase deciency": "G90",
  "Favism": "D55",
  "Glutathione synthetase deciency": "E72",
  "Sandho disease, infantile, juvenile, and adult forms": "E75",
  "3-methylglutaconicaciduria": "E71",
  "Fanconi anemia": "D61",
  "Usher syndrome": "H35",
  "Mitochondrial complex deciency": "E88",
  "Rubenstein-Taybi syndrome": "Q87",
  "Alexander disease": "G31",
  "Vohwinkel syndrome": "Q82",
  "Jervell and Lange-Nielsen syndrome": "I49",
  "Roussy-Levy syndrome": "G60",
  "Carney complex": "D48",
  "Occipital horn syndrome": "E76",
  "Williams-Beuren syndrome": "Q93",
  "Shprintzen-Goldberg syndrome": "Q87",
  "Simpson-Golabi-Behmel syndrome": "Q87",
  "Costello syndrome": "Q87",
  "Nijmegen breakage syndrome": "Q93",
  "Zlotogora-Ogur syndrome": "Q87",
  "Parkes Weber syndrome": "Q27",
  "Coffin-Lowry syndrome": "Q87",
  "Immunodysregulation, polyendocrinopathy, and enteropathy, X-linked": "D82",
  "Myotonia congenita": "G71",
  "Central core disease": "G71",
  "Cramps, potassium-aggravated": "G71",
  "Paramyotonia congenita": "G71",
  "Charcot-Marie-Tooth disease": "G60",
  "Leigh syndrome": "G31",
  "Subcortical laminar heterotopia": "Q04",
  "Branchiootic syndrome": "Q18",
  "Pallidopontonigral degeneration": "G23",
  "PPM-X syndrome": "Q98",
  "Norrie disease": "H35",
  "Pelizaeus-Merzbacher disease": "E75",
  "Gerstmann-Straussler disease": "A81",
  "Prion disease with protracted course": "A81",
  "Creatine deciency syndrome, X-linked": "E70",
  "PCWH": "Q87",
  "Leanness, inherited": "R63",
  "Leber congenital amaurosis": "H35",
  "Iridogoniodysgenesis": "Q15",
  "Wagner syndrome": "H35",
  "Axenfeld anomaly": "Q13",
  "Coats disease": "H35",
  "Foveal hypoplasia": "Q14",
  "Butterfly dystrophy, retinal": "H35",
  "Foveomacular dystrophy, adult-onset, with choroidal neovascularization": "H35",
  "Bothnia retinal dystrophy": "H35",
  "Liddle syndrome": "E26",
  "Denys-Drash syndrome": "Q87",
  "Central hypoventilation syndrome": "G47",
  "Pfeiffer syndrome": "Q87",
  "Parietal foramina": "Q75",
  "SMED Strudwick type": "Q77",
  "Thanatophoric dysplasia, types I and II": "Q77",
  "Larson syndrome": "Q74",
  "Vertical talus": "Q66",
  "Morning glory disc anomaly": "Q14",
  "Solitary median maxillary central incisor": "K00",
  "Beta-2-adrenoreceptor agonist, reduced response to": "R09",
  "Aquaporin-1 deficiency": "E88",
  "van Buchem disease": "M85"
}


In [48]:
matched, unmatched = [], []

for disease in df_unmapped["disease_name"].values.tolist():
    if dict_qwen[disease] != dict_gemma[disease]:
        print(f"Disease = {disease}, Qwen's verdict = {dict_qwen[disease]} vs Gemma's verdict = {dict_gemma[disease]}")
        unmatched.append(disease)
    else:
        matched.append(disease)

len(matched) + len(unmatched) == len(df_unmapped["disease_name"].values.tolist())

Disease = Lhermitte-Duclos syndrome, Qwen's verdict = D33 vs Gemma's verdict = Q04
Disease = Oligodontia-colorectal cancer syndrome, Qwen's verdict = Z15 vs Gemma's verdict = K00
Disease = Long QT syndrome, Qwen's verdict = I45 vs Gemma's verdict = I49
Disease = Acquired long QT syndrome, Qwen's verdict = I45 vs Gemma's verdict = I49
Disease = Wolff-Parkinson-White syndrome, Qwen's verdict = I45 vs Gemma's verdict = I49
Disease = Weissenbacher-Zweymuller syndrome, Qwen's verdict = Q77 vs Gemma's verdict = Q87
Disease = EBD, Qwen's verdict = Q81 vs Gemma's verdict = Q82
Disease = Piebaldism, Qwen's verdict = L81 vs Gemma's verdict = Q82
Disease = Unna-Thost disease, nonepidermolytic, Qwen's verdict = Q82 vs Gemma's verdict = L85
Disease = Netherton syndrome, Qwen's verdict = Q80 vs Gemma's verdict = Q82
Disease = Leprechaunism, Qwen's verdict = E34 vs Gemma's verdict = E74
Disease = Angiotensin I-converting enzyme, Qwen's verdict = Z83 vs Gemma's verdict = nan
Disease = Total iodide org

True

In [65]:
dict_gemma_stage2 = {
    "Lhermitte-Duclos syndrome": "Q04",
    "Oligodontia-colorectal cancer syndrome": "Q87",
    "Long QT syndrome": "I45",
    "Acquired long QT syndrome": "I45",
    "Wolff-Parkinson-White syndrome": "I45",
    "Weissenbacher-Zweymuller syndrome": "Q87",
    "EBD": "Q81",
    "Piebaldism": "L81",
    "Unna-Thost disease, nonepidermolytic": "Q82",
    "Netherton syndrome": "Q80",
    "Leprechaunism": "E34",
    "Angiotensin I-converting enzyme": np.nan,
    "Total iodide organication defect": "E03",
    "Hematopoiesis, cyclic": "D70",
    "Adult i phenotype": np.nan,
    "May-Hegglin anomaly": "D72",
    "Sebastian syndrome": "D69",
    "Pyropoikilocytosis": "D58",
    "IgE levels QTL": np.nan,
    "HDL cholesterol level QTL": np.nan,
    "Placental steroid sulfatase deciency": "E75",
    "Creatine phosphokinase": np.nan,
    "Dopamine beta-hydroxylase deciency": "G90",
    "Glutathione synthetase deciency": "D55",
    "Alexander disease": "G31",
    "Jervell and Lange-Nielsen syndrome": "I45",
    "Carney complex": "D44",
    "Occipital horn syndrome": "E83",
    "Nijmegen breakage syndrome": "Q87",
    "PPM-X syndrome": "F71",
    "Norrie disease": "Q15",
    "Creatine deciency syndrome, X-linked": "E72",
    "Leanness, inherited": "R63",
    "Iridogoniodysgenesis": "Q13",
    "Liddle syndrome": "I15",
    "Denys-Drash syndrome": "N04",
    "Pfeiffer syndrome": "Q87",
    "Beta-2-adrenoreceptor agonist, reduced response to": np.nan,
    "Aquaporin-1 deficiency": np.nan,
    "van Buchem disease": "M85",
}
dict_qwen_stage2 = {
  "Lhermitte-Duclos syndrome": "D33",
  "Oligodontia-colorectal cancer syndrome": "Q87",
  "Long QT syndrome": "I45",
  "Acquired long QT syndrome": "I45",
  "Wolff-Parkinson-White syndrome": "I45",
  "Weissenbacher-Zweymuller syndrome": "Q77",
  "EBD": "Q81",
  "Piebaldism": "L81",
  "Unna-Thost disease, nonepidermolytic": "Q82",
  "Netherton syndrome": "Q80",
  "Leprechaunism": "E34",
  "Angiotensin I-converting enzyme": "N/A",
  "Total iodide organification defect": "E03",
  "Hematopoiesis, cyclic": "D70",
  "Adult i phenotype": "N/A",
  "May-Hegglin anomaly": "D69",
  "Sebastian syndrome": "D69",
  "Pyropoikilocytosis": "D58",
  "IgE levels QTL": "N/A",
  "HDL cholesterol level QTL": "N/A",
  "Placental steroid sulfatase deficiency": "E70",
  "Creatine phosphokinase": "N/A",
  "Dopamine beta-hydroxylase deficiency": "G90",
  "Glutathione synthetase deficiency": "E72",
  "Alexander disease": "G31",
  "Jervell and Lange-Nielsen syndrome": "I45",
  "Carney complex": "D44",
  "Occipital horn syndrome": "E83",
  "Nijmegen breakage syndrome": "D82",
  "PPM-X syndrome": "F72",
  "Norrie disease": "Q15",
  "Creatine deficiency syndrome, X-linked": "E72",
  "Leanness, inherited": "N/A",
  "Iridogoniodysgenesis": "Q13",
  "Liddle syndrome": "I15",
  "Denys-Drash syndrome": "Q86",
  "Pfeiffer syndrome": "Q75",
  "Beta-2-adrenoreceptor agonist, reduced response to": "N/A",
  "Aquaporin-1 deficiency": "N/A",
  "van Buchem disease": "Q78",
}

In [66]:
len(unmatched)

40

In [67]:
matched_stage2, unmatched_stage2 = [], []

for disease in unmatched:
    if not (disease in dict_qwen_stage2 and disease in dict_gemma_stage2):
        unmatched_stage2.append(disease)
    elif dict_qwen_stage2[disease] != dict_gemma_stage2[disease]:
        print(f"Disease = {disease}, Qwen's verdict = {dict_qwen_stage2[disease]} vs Gemma's verdict = {dict_gemma_stage2[disease]}")
        unmatched_stage2.append(disease)
    else:
        matched_stage2.append(disease)

len(matched_stage2) + len(unmatched_stage2) == len(unmatched)

Disease = Lhermitte-Duclos syndrome, Qwen's verdict = D33 vs Gemma's verdict = Q04
Disease = Weissenbacher-Zweymuller syndrome, Qwen's verdict = Q77 vs Gemma's verdict = Q87
Disease = Angiotensin I-converting enzyme, Qwen's verdict = N/A vs Gemma's verdict = nan
Disease = Adult i phenotype, Qwen's verdict = N/A vs Gemma's verdict = nan
Disease = May-Hegglin anomaly, Qwen's verdict = D69 vs Gemma's verdict = D72
Disease = IgE levels QTL, Qwen's verdict = N/A vs Gemma's verdict = nan
Disease = HDL cholesterol level QTL, Qwen's verdict = N/A vs Gemma's verdict = nan
Disease = Creatine phosphokinase, Qwen's verdict = N/A vs Gemma's verdict = nan
Disease = Nijmegen breakage syndrome, Qwen's verdict = D82 vs Gemma's verdict = Q87
Disease = PPM-X syndrome, Qwen's verdict = F72 vs Gemma's verdict = F71
Disease = Leanness, inherited, Qwen's verdict = N/A vs Gemma's verdict = R63
Disease = Denys-Drash syndrome, Qwen's verdict = Q86 vs Gemma's verdict = N04
Disease = Pfeiffer syndrome, Qwen's ver

True

In [69]:
len(unmatched_stage2), unmatched_stage2

(21,
 ['Lhermitte-Duclos syndrome',
  'Weissenbacher-Zweymuller syndrome',
  'Angiotensin I-converting enzyme',
  'Total iodide organication defect',
  'Adult i phenotype',
  'May-Hegglin anomaly',
  'IgE levels QTL',
  'HDL cholesterol level QTL',
  'Placental steroid sulfatase deciency',
  'Creatine phosphokinase',
  'Dopamine beta-hydroxylase deciency',
  'Glutathione synthetase deciency',
  'Nijmegen breakage syndrome',
  'PPM-X syndrome',
  'Creatine deciency syndrome, X-linked',
  'Leanness, inherited',
  'Denys-Drash syndrome',
  'Pfeiffer syndrome',
  'Beta-2-adrenoreceptor agonist, reduced response to',
  'Aquaporin-1 deficiency',
  'van Buchem disease'])

In [73]:
add_matching = dict() 

for el in matched:
    add_matching[el] = dict_qwen[el]

for el in matched_stage2:
    add_matching[el] = dict_qwen_stage2[el]

len(add_matching)

100

In [82]:
add_matching

{'Dentinogenesis imperfecta, Shields type': 'K00',
 'Glioblastoma': 'C71',
 'Wilms tumor': 'C64',
 'Rhabdomyosarcoma': 'C49',
 'Turcot syndrome': 'Q85',
 'Squamous cell carcinoma': 'C44',
 'Pheochromocytoma': 'C74',
 'Cancer susceptibility': 'Z15',
 'Cafe-au-lait spots': 'L81',
 'Li-Fraumeni syndrome': 'Z15',
 'Gardner syndrome': 'D12',
 'Li Fraumeni syndrome': 'Z15',
 'Lynch cancer family syndrome II': 'Z15',
 'Merkel cell carcinoma': 'C4A',
 'Peutz-Jeghers syndrome': 'Q85',
 'Brugada syndrome': 'I49',
 'Noncompaction of left ventricular myocardium': 'I42',
 'Weill-Marchesani syndrome': 'Q87',
 'Self-healing collodion baby': 'Q80',
 'Birt-Hogg-Dube syndrome': 'Q87',
 'Peters anomaly': 'Q13',
 'Angelman syndrome': 'Q93',
 'MODY': 'E13',
 'Growth hormone': 'E23',
 'Goiter': 'E04',
 'Sea-blue histiocyte disease': 'D76',
 'Coumarin resistance': 'D68',
 'Complementary component deficiency': 'D84',
 'Atopy': 'L20',
 'Adenosine deaminase deficiency': 'D81',
 'Graft-versus-host disease': 'T86

In [74]:
df_unmapped.head(2)

,disease_id,disease_name,disease_class,icd10_codes,primary_icd10,mapping_method,similarity_score
0,406,"Dentinogenesis imperfecta, Shields type",Bone,NaN,NaN,unmapped,NaN
1,384,Glioblastoma,Cancer,NaN,NaN,unmapped,NaN


In [75]:
df_unmapped["icd10_codes"].isna().sum()

np.int64(121)

In [85]:
mask = df_unmapped['disease_name'].isin(add_matching.keys())

df_unmapped.loc[mask, 'icd10_codes'] = df_unmapped['disease_name'].map(add_matching)
df_unmapped.loc[mask, 'primary_icd10'] = df_unmapped['disease_name'].map(add_matching)
df_unmapped.loc[mask, 'mapping_method'] = "manual_qwen_gemma"

df_unmapped["icd10_codes"].isna().sum()

np.int64(21)

In [86]:
df_unmapped.head()

,disease_id,disease_name,disease_class,icd10_codes,primary_icd10,mapping_method,similarity_score
0,406,"Dentinogenesis imperfecta, Shields type",Bone,K00,K00,manual_qwen_gemma,NaN
1,384,Glioblastoma,Cancer,C71,C71,manual_qwen_gemma,NaN
2,203,Wilms tumor,Cancer,C64,C64,manual_qwen_gemma,NaN
3,497,Rhabdomyosarcoma,Cancer,C49,C49,manual_qwen_gemma,NaN
4,118,Turcot syndrome,Cancer,Q85,Q85,manual_qwen_gemma,NaN


In [91]:
add_matching["Dentinogenesis imperfecta, Shields type"]

'K00'

In [92]:
df_names_all = pd.concat([df_mapped, df_unmapped])
df_mapped.shape[0] + df_unmapped.shape[0] == df_names_all.shape[0]

True

In [94]:
df_names_all.to_csv("disease_icd10_mapping_ALL.csv", index=False)

In [95]:
df_adj

,340,858,341,877,878,876,454,874,875,872,...,948,947,579,578,577,576,575,1181,46,471
340,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
858,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
341,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
877,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
878,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [7]:
import pandas as pd
import numpy as np

In [8]:
df_names_all = pd.read_csv("disease_icd10_mapping_ALL.csv")
df_names_all

,disease_id,disease_name,disease_class,icd10_codes,primary_icd10,mapping_method,similarity_score
0,222,Osteoporosis,Bone,M82; M81; M80,M82,exact_match,NaN
1,306,Epiphyseal dysplasia,Bone,Q77,Q77,exact_match,NaN
2,277,Osteopetrosis,Bone,Q78,Q78,exact_match,NaN
3,286,Osteogenesis imperfecta,Bone,Q78,Q78,exact_match,NaN
4,288,"Achondrogenesis-hypochondrogenesis, type II",Bone,Q77,Q77,exact_match,NaN
...,...,...,...,...,...,...,...
632,877,Morning glory disc anomaly,Skeletal,Q14,Q14,manual_qwen_gemma,NaN
633,991,Solitary median maxillary central incisor,Skeletal,K00,K00,manual_qwen_gemma,NaN
634,69,"Beta-2-adrenoreceptor agonist, reduced respons...",Unclassified,NaN,NaN,unmapped,NaN
635,134,Aquaporin-1 deficiency,Unclassified,NaN,NaN,unmapped,NaN


In [9]:
df_names_all["disease_id"].nunique(), df_names_all["disease_name"].nunique()

(516, 515)

In [10]:
df_names_all[(df_names_all["icd10_codes"].isna()) & (~df_names_all["primary_icd10"].isna())]

,disease_id,disease_name,disease_class,icd10_codes,primary_icd10,mapping_method,similarity_score


In [11]:
df_names_all[(~df_names_all["icd10_codes"].isna()) & (df_names_all["primary_icd10"].isna())]

,disease_id,disease_name,disease_class,icd10_codes,primary_icd10,mapping_method,similarity_score


In [12]:
disease_names_mapping = df_names_all[(~df_names_all["icd10_codes"].isna()) & (~df_names_all["disease_name"].isna())]
disease_names_mapping

,disease_id,disease_name,disease_class,icd10_codes,primary_icd10,mapping_method,similarity_score
0,222,Osteoporosis,Bone,M82; M81; M80,M82,exact_match,NaN
1,306,Epiphyseal dysplasia,Bone,Q77,Q77,exact_match,NaN
2,277,Osteopetrosis,Bone,Q78,Q78,exact_match,NaN
3,286,Osteogenesis imperfecta,Bone,Q78,Q78,exact_match,NaN
4,288,"Achondrogenesis-hypochondrogenesis, type II",Bone,Q77,Q77,exact_match,NaN
...,...,...,...,...,...,...,...
629,479,"Thanatophoric dysplasia, types I and II",Skeletal,Q77,Q77,manual_qwen_gemma,NaN
630,502,Larson syndrome,Skeletal,Q74,Q74,manual_qwen_gemma,NaN
631,627,Vertical talus,Skeletal,Q66,Q66,manual_qwen_gemma,NaN
632,877,Morning glory disc anomaly,Skeletal,Q14,Q14,manual_qwen_gemma,NaN


In [13]:
disease_names_mapping["disease_id"].nunique()

495

In [14]:
import networkx as nx

In [15]:
G = nx.read_edgelist("diseasome.edgelist")
df_edge_list = nx.to_pandas_edgelist(G)
df_edge_list

,source,target
0,340,858
1,340,341
2,858,1285
3,341,877
4,341,878
...,...,...
1183,578,576
1184,578,575
1185,577,576
1186,577,575


In [16]:
disease_names_mapping["disease_id"] = disease_names_mapping["disease_id"].astype(str)

C:\Users\Asus\AppData\Local\Temp\ipykernel_11336\3675361506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disease_names_mapping["disease_id"] = disease_names_mapping["disease_id"].astype(str)


In [17]:
mapped_pairs = []

for i, row in df_edge_list.iterrows():
    source = row["source"]
    target = row["target"]

    mapped_source = disease_names_mapping["icd10_codes"][disease_names_mapping["disease_id"] == source]
    mapped_target = disease_names_mapping["icd10_codes"][disease_names_mapping["disease_id"] == target]

    if len(mapped_source) > 0 and len(mapped_target) > 0:
        mapped_source = mapped_source.values[0].split("; ")
        mapped_target = mapped_target.values[0].split("; ")    
    
        for src in mapped_source:
            for tgt in mapped_target:
                mapped_pairs.append(
                    {
                        "source": src,
                        "target": tgt,
                    }
                )
        if len(mapped_source) > 1:
            for i in range(len(mapped_source) - 1):
                for j in range(i+1, len(mapped_source)):
                    mapped_pairs.append(
                        {
                            "source": mapped_source[i],
                            "target": mapped_source[j],
                        }
                    )
        if len(mapped_target) > 1:
            for i in range(len(mapped_target) - 1):
                for j in range(i+1, len(mapped_target)):
                    mapped_pairs.append(
                        {
                            "source": mapped_target[i],
                            "target": mapped_target[j],
                        }
                    )

len(mapped_pairs)

6294

In [18]:
genes_graph = pd.DataFrame(mapped_pairs)
genes_graph

,source,target
0,H42,H48
1,H42,H47
2,Q15,H48
3,Q15,H47
4,H40,H48
...,...,...
6289,E20,P27
6290,E20,E22
6291,N85,P27
6292,N85,E22


In [19]:
final_matrix = nx.to_pandas_adjacency(nx.from_pandas_edgelist(genes_graph))
final_matrix

,H42,H48,H47,Q15,H40,Q13,E71,Q14,H19,H16,...,E70,E04,E35,J63,Q98,T15,P72,Q18,T82,I68
H42,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H48,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H47,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Q15,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H40,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T15,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Q18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T82,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [20]:
final_matrix.to_csv("./matrices_new/genes_graph_paper_2007.csv")
final_matrix.to_csv("./matrices_new/relevant_matrices/genes_graph_paper_2007.csv")
final_matrix.to_csv("./matrices_new/relevant_matrices_closed_data/genes_graph_paper_2007.csv")

In [28]:
final_matrix.values.sum(), round(final_matrix.values.sum() / final_matrix.shape[0] / final_matrix.shape[1] * 100, 3)

(5276.0, 3.416)

In [29]:
final_matrix.values.min(), final_matrix.values.max()

(0.0, 1.0)